This notebook provides a convenient wrapper for bag_reader.  
It looks for all bagfiles in a specified directory and converts to pickle/matfile for further processing.

* [ ] TODO: Filter out "No intention pressed"
* [ ] TODO: The intention should be included only when it happens at the current time step, undetermined otherwise

In [1]:
import glob
import pickle
import scipy.io as sio
from bag_reader import *

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.
/home/mpc/carla/PythonAPI/analysis/transformations.py:1926: UserWarning: No module named '_transformations'
  warnings.warn(str(err))


In [2]:
bagfiles_to_process = glob.glob('../examples/bags/*.bag')

print('Found %d bagfiles: %s' % (len(bagfiles_to_process), bagfiles_to_process))


Found 59 bagfiles: ['../examples/bags/parking_p3_t2_e8_2019-11-26-16-14-26.bag', '../examples/bags/parking_p2_t1_e3_2019-11-04-16-33-55.bag', '../examples/bags/parking_p1_t1_e5_2019-11-04-14-36-41.bag', '../examples/bags/parking_p2_t2_e9_2019-11-04-16-42-32.bag', '../examples/bags/parking_p3_t2_e7_2019-11-26-16-13-59.bag', '../examples/bags/parking_p2_t1_e4_2019-11-04-16-34-11.bag', '../examples/bags/parking_p3_t2_e9_2019-11-26-16-14-52.bag', '../examples/bags/parking_p3_t1_e8_2019-11-26-15-47-41.bag', '../examples/bags/parking_p3_t1_e2_2019-11-26-15-45-40.bag', '../examples/bags/parking_p3_t1_e9_2019-11-26-15-48-06.bag', '../examples/bags/parking_p2_t2_e1_2019-11-04-16-37-56.bag', '../examples/bags/parking_p1_t1_e3_2019-11-04-14-35-59.bag', '../examples/bags/parking_p3_t1_e1_2019-11-26-15-45-22.bag', '../examples/bags/parking_p1_t1_e7_2019-11-04-14-37-31.bag', '../examples/bags/parking_p2_t2_e7_2019-11-04-16-41-19.bag', '../examples/bags/parking_p2_t2_e2_2019-11-04-16-38-43.bag', '../

### Topics processed by bag_reader.py

* [x] /carla/actor_list
* [x] /carla/hero/collision
* [x] /carla/hero/gnss/front/fix
* [x] /carla/hero/objects
* [x] /carla/hero/odometry
* [x] /carla/hero/vehicle_info
* [x] /carla/hero/vehicle_status
* [x] /carla/odometry
* [x] /carla/status
* [x] /carla/world_info
* [x] /intention
* [ ] <span style="color:blue"> /carla/objects </span>
* [ ] <span style="color:blue"> /rosout </span>
* [ ] <span style="color:blue"> /rosout_agg </span>
* [ ] <span style="color:blue"> /tf </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/camera_info </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/image_color </span>
* [ ] <span style="color:blue"> /carla/marker </span>
* [ ] <span style="color:blue"> /clock </span>

### Additional Notes
<span style="color:blue"> We do not plan to process blue topics (or will do it separately for images). /carla/objects contains the ego vehicle as well, so excluded. </span>

<span style="color:red;font-weight:bold"> Outlier detection is currently only applied for the ego odometry.</span>


In [3]:
for bagname in bagfiles_to_process:
    print('Processing ', bagname)
    result_dict = process_bag(bagname)
    
    fname = bagname.split('.bag')[0]
    assert('parking' in fname)
    
    '''
    # Save to file (matfile, needs work)
    fname += '.mat'
    print('Saving ', fname)
    sio.savemat(fname, result_dict)
    '''
    
    fname += '.pkl'
    pickle.dump(result_dict, open(fname, 'wb'))
    
    print('Finished :', fname)

Processing  ../examples/bags/parking_p3_t2_e8_2019-11-26-16-14-26.bag
Finished : ../examples/bags/parking_p3_t2_e8_2019-11-26-16-14-26.pkl
Processing  ../examples/bags/parking_p2_t1_e3_2019-11-04-16-33-55.bag
Finished : ../examples/bags/parking_p2_t1_e3_2019-11-04-16-33-55.pkl
Processing  ../examples/bags/parking_p1_t1_e5_2019-11-04-14-36-41.bag
Finished : ../examples/bags/parking_p1_t1_e5_2019-11-04-14-36-41.pkl
Processing  ../examples/bags/parking_p2_t2_e9_2019-11-04-16-42-32.bag
Finished : ../examples/bags/parking_p2_t2_e9_2019-11-04-16-42-32.pkl
Processing  ../examples/bags/parking_p3_t2_e7_2019-11-26-16-13-59.bag
Finished : ../examples/bags/parking_p3_t2_e7_2019-11-26-16-13-59.pkl
Processing  ../examples/bags/parking_p2_t1_e4_2019-11-04-16-34-11.bag
	Outlier detected.  Setting final time 635.390 vs. last entry at time 635.557 for ego odom.
Finished : ../examples/bags/parking_p2_t1_e4_2019-11-04-16-34-11.pkl
Processing  ../examples/bags/parking_p3_t2_e9_2019-11-26-16-14-52.bag
Fini

Finished : ../examples/bags/parking_p1_t1_e8_2019-11-04-14-37-56.pkl
Processing  ../examples/bags/parking_p3_t2_e1_2019-11-26-16-11-56.bag
	Outlier detected.  Setting final time 279.387 vs. last entry at time 279.420 for ego odom.
Finished : ../examples/bags/parking_p3_t2_e1_2019-11-26-16-11-56.pkl
Processing  ../examples/bags/parking_p1_t1_e1_2019-11-04-14-34-58.bag
Finished : ../examples/bags/parking_p1_t1_e1_2019-11-04-14-34-58.pkl
Processing  ../examples/bags/parking_p1_t2_e5_2019-11-04-14-43-12.bag
Finished : ../examples/bags/parking_p1_t2_e5_2019-11-04-14-43-12.pkl
Processing  ../examples/bags/parking_p2_t2_e3_2019-11-04-16-39-13.bag
	Outlier detected.  Setting final time 1358.957 vs. last entry at time 1359.057 for ego odom.
Finished : ../examples/bags/parking_p2_t2_e3_2019-11-04-16-39-13.pkl
